___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 8: Unsupervised learning

## Lab work 8



#### Мета: 

* навчитися використовувати моделі без учителем

### Завдання 1:

Опрацюйте файл `Groceries.csv`. Використовуючи алгоритм асоціативних зв'язків, знайдіть:
- зробіть описову статистику даного файлу
- ведіть топ-3 продукти, що продаються
- виведіть топ-2 пари продуктів, що продаються
- придумайте, як отриману інформацію можна використатти (опишіть в 3-4 реченнях)

In [1]:
import pandas as pd
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt

In [2]:
gr_df = pd.read_csv(r'D:\Работа ВВЮ\DataScience\groceries.csv')

### зробіть описову статистику даного файлу

In [3]:
gr_df=gr_df.drop('Item(s)', axis = 1)
gr_df.head()

,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,Item 10,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
gr_df.shape

(9835, 32)

In [5]:
gr_df.isnull().sum()

Item 1        0
Item 2     2159
Item 3     3802
Item 4     5101
Item 5     6106
Item 6     6961
Item 7     7606
Item 8     8151
Item 9     8589
Item 10    8939
Item 11    9185
Item 12    9367
Item 13    9484
Item 14    9562
Item 15    9639
Item 16    9694
Item 17    9740
Item 18    9769
Item 19    9783
Item 20    9797
Item 21    9806
Item 22    9817
Item 23    9821
Item 24    9827
Item 25    9828
Item 26    9828
Item 27    9829
Item 28    9830
Item 29    9831
Item 30    9834
Item 31    9834
Item 32    9834
dtype: int64

In [6]:
items = set()
for col in gr_df:
    items.update(gr_df[col].unique())
print(items)

{nan, 'hamburger meat', 'frozen fish', 'waffles', 'pork', 'decalcifier', 'dishes', 'cocoa drinks', 'canned fruit', 'fruit/vegetable juice', 'canned fish', 'soda', 'dog food', 'hair spray', 'pasta', 'seasonal products', 'turkey', 'skin care', 'beef', 'frozen chicken', 'hygiene articles', 'zwieback', 'candles', 'fish', 'sweet spreads', 'mustard', 'nut snack', 'sausage', 'specialty chocolate', 'sliced cheese', 'curd cheese', 'yogurt', 'domestic eggs', 'rubbing alcohol', 'artif. sweetener', 'grapes', 'organic products', 'house keeping products', 'whipped/sour cream', 'pip fruit', 'jam', 'popcorn', 'liquor', 'butter milk', 'specialty fat', 'processed cheese', 'flower (seeds)', 'bottled water', 'coffee', 'rum', 'kitchen towels', 'ketchup', 'frozen meals', 'cat food', 'vinegar', 'white wine', 'spread cheese', 'UHT-milk', 'chocolate', 'semi-finished bread', 'mayonnaise', 'herbs', 'photo/film', 'specialty vegetables', 'onions', 'ready soups', 'canned vegetables', 'condensed milk', 'red/blush wi

In [60]:
#Уникальные товары в наборе:
items = {x for x in items if pd.notna(x)}
print(items)

{'hamburger meat', 'frozen fish', 'waffles', 'pork', 'decalcifier', 'dishes', 'cocoa drinks', 'canned fruit', 'fruit/vegetable juice', 'canned fish', 'soda', 'hair spray', 'dog food', 'pasta', 'seasonal products', 'turkey', 'skin care', 'beef', 'frozen chicken', 'hygiene articles', 'zwieback', 'candles', 'fish', 'sweet spreads', 'mustard', 'nut snack', 'sausage', 'specialty chocolate', 'sliced cheese', 'curd cheese', 'yogurt', 'domestic eggs', 'rubbing alcohol', 'artif. sweetener', 'grapes', 'organic products', 'house keeping products', 'whipped/sour cream', 'pip fruit', 'jam', 'popcorn', 'liquor', 'butter milk', 'specialty fat', 'processed cheese', 'flower (seeds)', 'bottled water', 'coffee', 'rum', 'kitchen towels', 'ketchup', 'frozen meals', 'cat food', 'vinegar', 'white wine', 'spread cheese', 'UHT-milk', 'chocolate', 'semi-finished bread', 'mayonnaise', 'herbs', 'photo/film', 'specialty vegetables', 'onions', 'ready soups', 'canned vegetables', 'condensed milk', 'red/blush wine', 

In [61]:
print('Всего уникальных товаров:' ,len(items))

Всего уникальных товаров: 169


In [9]:
itemset = set(items)
encoded_vals = []
for index, row in gr_df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]
ohe_df = pd.DataFrame(encoded_vals)

In [15]:
ohe_df.head()

,hamburger meat,frozen fish,waffles,pork,decalcifier,dishes,cocoa drinks,canned fruit,fruit/vegetable juice,canned fish,...,toilet cleaner,cling film/bags,roll products,liver loaf,rolls/buns,specialty bar,semi-finished bread,citrus fruit,ready soups,margarine
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
#Считаем количество товаров в DataFrame
items_list = list(items)
dict_l = {}
for el in items_list:
    var = 0
    var = ohe_df[el].sum()
    dict_l[el] = var

product = pd.DataFrame.from_dict(dict_l, orient='index',
                       columns=['amount']) 

product=product.reset_index().sort_values('amount', ascending = False)

#### виведіть топ-3 продукти, що продаються

In [59]:
product.head(3)

,index,amount
94,whole milk,2513
126,other vegetables,1903
167,rolls/buns,1809


#### Используем библиотеку mlxtend для поиска взаимосвязи с товарами

In [76]:
freq_items = apriori(ohe_df, min_support=0.05, use_colnames=True, verbose=1)
freq_items.head(7).sort_values('support', ascending = False)

Processing 9 combinations | Sampling itemset size 3


,support,itemsets
2,0.174377,(soda)
5,0.139502,(yogurt)
4,0.093950,(sausage)
1,0.072293,(fruit/vegetable juice)
6,0.063447,(domestic eggs)
0,0.057651,(pork)
3,0.052466,(beef)


#### Выводим наиболее связанные товары мыжду собой

In [75]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.01)
rules.head(3).sort_values('confidence', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(yogurt),(whole milk),0.139502,0.255516,0.056024,0.401603,1.571735,0.020379,1.244132
2,(other vegetables),(whole milk),0.193493,0.255516,0.074835,0.386758,1.513634,0.025394,1.214013
0,(whole milk),(yogurt),0.255516,0.139502,0.056024,0.219260,1.571735,0.020379,1.102157


#### Вместо вывода:

На основе анализа ассоциативных связей можно оптимизировать набор товаров в магазине. Больше и чаще закупать йогурт, овощи и молоко, возможно расширить их ассортимент. Возможно стоит проанализировать, что еще покупается с основными наиболее популярными продуктами, тем самым расширить представленный ассортимент, возможно сделать перестановку полок с продуктами между собой

### Завдання 2:

Завантажте файл https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python
 
- Спробуйте провести кластеризацію, тобто виділити цільові групи, для яких можна впроваджувати певні маркетингові акції
- Опишіть отримані результати

In [101]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [78]:
df_c = pd.read_csv(r'D:\Работа ВВЮ\DataScience\Mall_Customers.csv')
df_c.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [79]:
df_c.shape

(200, 5)

In [90]:
df_c.set_index('CustomerID')

,Gender,Age,Annual Income (k$),Spending Score (1-100)
CustomerID,,,,
1,Male,19,15,39
2,Male,21,15,81
3,Female,20,16,6
4,Female,23,16,77
5,Female,31,17,40
...,...,...,...,...
196,Female,35,120,79
197,Female,45,126,28
198,Male,32,126,74


In [109]:
features = df_c[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']]


In [110]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)



In [111]:
from sklearn.cluster import KMeans
kmeans = KMeans(init="random", n_clusters=3, n_init=10, max_iter=300, random_state=42)

In [112]:
kmeans.fit(scaled_features)

KMeans(init='random', n_clusters=3, random_state=42)

In [113]:
kmeans.inertia_

295.26029648861186

In [114]:
kmeans.cluster_centers_

array([[-0.93381128, -0.67979753,  0.1338202 ],
       [-0.44315931,  1.03973525,  1.17726383],
       [ 0.8828863 ,  0.05040024, -0.61076442]])

In [115]:
kmeans.n_iter_

4